In [1]:
# 硬件上训练鉴别器
from mindquantum.core import Circuit, apply
from cqlib import TianYanPlatform, QuantumLanguage
from copy import deepcopy
import json
import numpy as np
import mindspore as ms
from mindspore import nn
from mindspore.common.parameter import Parameter

def update(dict0:dict, dict1:dict):
    """合并两个字典"""
    return {**dict0, **dict1}

def compute_cost(result:dict):
    res = json.loads(result['probability'])
    items = ['0111', '1011', '1101', '1110', '1010', '0101']
    cost = sum([res.get(item, 0) for item in items])
    return cost
    
def hardware_run(circs:list[Circuit], shots:int=1000):
    """向云平台批量提交线路信息, 并获取返回结果"""
    tasks = [circ.to_qcis(parametric=False) for circ in circs]
    query_list = platform.submit_experiment(circuit=tasks,
                                            language=QuantumLanguage.QCIS,
                                            num_shots=shots)
    results = platform.query_experiment(query_id=query_list,
                                            max_wait_time=60000,
                                            sleep_time=2)
    costs = []
    for i in range(len(circs)):
        costs.append(compute_cost(results[i]))
    return costs

def get_cost_with_grads(circuit:Circuit, values_a:np.ndarray):
    """根据参数移位法计算当前参数的梯度"""
    names_e = circuit.encoder_params_name
    names_a = circuit.ansatz_params_name
    h = np.pi / 2
    def grad_ops(values_e:np.ndarray):
        """values_e: encoder 的参数; values_a: ansatz 的参数"""
        circs = []
        for value_e in values_e:
            pr_e = dict(zip(names_e, value_e))
            pr_0 = update(pr_e, dict(zip(names_a, values_a)))
            circs.append(circuit.apply_value(pr_0)) # 前 batch_size 个线路为求期望值数据
        # 格式为:
        # a1p, a1n, a2p, a2n ... 样本 0
        # a1p, a1n, a2p, a2n ... 样本 1
        # ...
        for value_e in values_e:
            # 计算参数移位法所需的损失函数值
            pr_e = dict(zip(names_e, value_e))
            for i in range(len(values_a)):
                values_a_p = deepcopy(values_a)
                values_a_n = deepcopy(values_a)

                values_a_p[i] += h
                pr_a_p = update(pr_e, dict(zip(names_a, values_a_p)))
                circs.append(circuit.apply_value(pr_a_p))

                values_a_n[i] -= h
                pr_a_n = update(pr_e, dict(zip(names_a, values_a_n)))
                circs.append(circuit.apply_value(pr_a_n))

        costs = hardware_run(circs)
        cost = np.mean(costs[:batch_size])
        costs = costs[batch_size:]
        costs = ms.Tensor(costs).reshape(batch_size, -1)
        cost_p = costs[:,0::2].mean(axis=0)
        cost_n = costs[:,1::2].mean(axis=0)
        grads = (cost_p - cost_n) / 2
        return cost, grads
    return grad_ops

# 简单编码线路  
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 鉴别器
disc_circ = Circuit()
disc_circ.h(0).z(2, 0).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz("theta2",1).rz("theta3",3).h(1).h(3)

circ = encoder + disc_circ
circ.measure_all()

physical_qubits = [10, 16, 4, 11] # 比特映射
circ = apply(circ, physical_qubits)

login_key = "lABSAqU6oBZCM9wc6owgpVQ0Jdnjr4JkQZXnHjMxck0="
machine_name="tianyan176-2"
platform = TianYanPlatform(login_key=login_key, 
                           machine_name=machine_name)

theta = Parameter(ms.Tensor([0., 0., 0., 0.]), name="theta") 
optim = nn.Adam([theta], learning_rate=0.1)

steps_num = 51
batch_size = 4
alphas = np.random.rand(steps_num, batch_size, len(encoder.params_name)) * np.pi * 2

min_cost = 1
costs = []
for i in range(steps_num):
    grad_ops = get_cost_with_grads(circ, theta) # type:ignore
    cost, grads = grad_ops(ms.Tensor(alphas[i]))
    if cost < min_cost:
        min_cost = cost
        best_disc_weight = theta.asnumpy()
    print("step:", i, "theta", theta.asnumpy(), "cost:", cost, "grads:", grads.asnumpy())
    costs.append(cost)
    optim((grads,))

print(list(range(steps_num)))
print(costs)
print(min_cost)
print(best_disc_weight) 

step: 0 theta [0. 0. 0. 0.] cost: 0.047 grads: [-0.02575    -0.01837499  0.014625   -0.002875  ]
step: 1 theta [ 0.09999877  0.09999829 -0.09999784  0.099989  ] cost: 0.0345 grads: [-0.005625   -0.009875   -0.00274999  0.01425   ]
step: 2 theta [ 0.18134668  0.19425239 -0.15208927  0.0402905 ] cost: 0.03775 grads: [-0.0055   -0.011875 -0.013375 -0.01425 ]
step: 3 theta [ 0.25596133  0.28918046 -0.13929985  0.05207307] cost: 0.03425 grads: [ 0.01525   0.000125 -0.009125  0.00925 ]
step: 4 theta [ 0.28045443  0.36663604 -0.10563589  0.03680326] cost: 0.041499999999999995 grads: [ 0.000125    0.018875   -0.003      -0.00300001]
step: 5 theta [ 0.3009358   0.3843141  -0.07006101  0.03127349] cost: 0.0415 grads: [ 0.00775  0.02825 -0.00425 -0.005  ]
step: 6 theta [ 0.30534756  0.36000466 -0.03011248  0.03793562] cost: 0.03875 grads: [0.018875 0.014375 0.007375 0.0015  ]
step: 7 theta [ 0.28282806  0.32338408 -0.01269649  0.04045461] cost: 0.043250000000000004 grads: [ 0.02075   0.00575   0.

In [5]:
# 验证训练好的鉴别器的效果
from mindquantum.core import Circuit, apply
from cqlib import TianYanPlatform, QuantumLanguage
import json
import numpy as np

def update(dict0:dict, dict1:dict):
    """合并两个字典"""
    return {**dict0, **dict1}

def compute_cost(result:dict):
    res = json.loads(result['probability'])
    items = ['0111', '1011', '1101', '1110', '1010', '0101']
    cost = sum([res.get(item, 0) for item in items])
    return cost
    
def hardware_run(circs:list[Circuit], shots:int=1000):
    """向云平台批量提交线路信息, 并获取返回结果"""
    tasks = [circ.to_qcis(parametric=False) for circ in circs]
    query_list = platform.submit_experiment(circuit=tasks,
                                            language=QuantumLanguage.QCIS,
                                            num_shots=shots)
    results = platform.query_experiment(query_id=query_list,
                                            max_wait_time=60000,
                                            sleep_time=2)
    costs = []
    for i in range(len(circs)):
        costs.append(compute_cost(results[i]))
    return costs

def func_cost(circuit:Circuit, values_a):
    """求给定 ansatz 参数下，不同 encoder 参数下的损失函数值, values_a: ansatz 的参数"""
    names_e = circuit.encoder_params_name
    names_a = circuit.ansatz_params_name
    pr_a = dict(zip(names_a, values_a))
    circuit = circuit.apply_value(pr_a)
    
    def sampling(values_e):
        """values_e: encoder 的参数"""
        circs = []
        for values in values_e:
            pr_e = dict(zip(names_e, values))
            circs.append(circuit.apply_value(pr_e))
        costs = hardware_run(circs)
        return np.array(costs).mean()
    return sampling

# 简单编码线路  
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 鉴别器
disc_circ = Circuit()
disc_circ.h(0).z(2, 0).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz("theta2",1).rz("theta3",3).h(1).h(3)

circ = encoder + disc_circ
circ.measure_all()

physical_qubits = [10, 16, 4, 11] # 比特映射
circ = apply(circ, physical_qubits)

login_key = "lABSAqU6oBZCM9wc6owgpVQ0Jdnjr4JkQZXnHjMxck0="
machine_name="tianyan176-2"
platform = TianYanPlatform(login_key=login_key, 
                           machine_name=machine_name)

# 用训练好的参数
theta = np.array([0.1692159, 0.1617644, -0.11142179, 0.05916444])
sampling = func_cost(circ, theta) # type:ignore

sample_num = 100
alphas = np.random.rand(sample_num, len(encoder.params_name)) * np.pi * 2
cost_0 = sampling(alphas[:50])
cost_1 = sampling(alphas[50:])
print((cost_0 + cost_1) / 2)
 
# 用全零参数
theta = np.array([0., 0., 0., 0.])
sampling = func_cost(circ, theta) # type:ignore
cost_0 = sampling(alphas[:50])
cost_1 = sampling(alphas[50:])
print((cost_0 + cost_1) / 2)


0.03931000000000001
0.045520000000000005


In [7]:
# 非参数化量子线路等价性验证（一个参数门）

from mindquantum import Circuit, apply
import numpy as np
np.random.seed(1)

def compute_cost(result:dict):
    res = json.loads(result['probability'])
    items = ['0111', '1011', '1101', '1110', '1010', '0101']
    cost = sum([res.get(item, 0) for item in items])
    return cost

def hardware_run(circs:list[Circuit], shots:int=1000):
    """向云平台批量提交线路信息, 并获取返回结果"""
    tasks = [circ.to_qcis(parametric=False) for circ in circs]
    query_list = platform.submit_experiment(circuit=tasks,
                                            language=QuantumLanguage.QCIS,
                                            num_shots=shots)
    results = platform.query_experiment(query_id=query_list,
                                            max_wait_time=60000,
                                            sleep_time=2)
    costs = []
    for i in range(len(circs)):
        costs.append(compute_cost(results[i]))
    return costs

def func_cost(circuit:Circuit, value_delta):
    """求给定 ansatz 参数下，不同 encoder 参数下的损失函数值, values_a: ansatz 的参数"""
    names_e = circuit.encoder_params_name
    pr = {"delta":value_delta}
    circuit = circuit.apply_value(pr)
    
    def sampling(values_e):
        """values_e: encoder 的参数"""
        circs = []
        for values in values_e:
            pr_e = dict(zip(names_e, values))
            circs.append(circuit.apply_value(pr_e))
        costs = hardware_run(circs)
        return np.array(costs).mean()
    return sampling

# 简单编码线路
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 参考线路
ref_circ = Circuit()
ref_circ.x(1).z(1, 0).x(1) 
ref_circ.x(0).z(1, 0).x(0)
ref_circ.z(1, 0)

# 生成器线路
gen_circ = Circuit().x(2).x(3).z(3,2).x(2).x(3).ry("delta", 2)

disc_weight = np.array([0.1692159, 0.1617644, -0.11142179, 0.05916444]) # 一个训练好的实例

disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ # type_ignore
circ.measure_all()
physical_qubits = [10, 16, 4, 11] # 比特映射
circ = apply(circ, physical_qubits)

login_key = "lABSAqU6oBZCM9wc6owgpVQ0Jdnjr4JkQZXnHjMxck0="
machine_name="tianyan176-2"
platform = TianYanPlatform(login_key=login_key, 
                           machine_name=machine_name)

# 随机量子态用于评估
sample_num = 100
alphas = np.random.rand(sample_num, len(encoder.params_name)) * np.pi * 2
deltas = []
costs = []
for i in range(11):
    delta = i * 2 * np.pi / 10
    deltas.append(delta)
    sampling = func_cost(circ, delta) # type:ignore
    cost_0 = sampling(alphas[:50])
    cost_1 = sampling(alphas[50:])
    cost = (cost_0 + cost_1) / 2
    costs.append(cost)
    print(i, delta, cost)

print("deltas:\n", deltas)
print("costs:\n", costs)

0 0.0 0.09408
1 0.6283185307179586 0.12778
2 1.2566370614359172 0.20297
3 1.8849555921538759 0.29556
4 2.5132741228718345 0.36757000000000006
5 3.141592653589793 0.3901300000000001
6 3.7699111843077517 0.36123000000000005
7 4.39822971502571 0.29033
8 5.026548245743669 0.19801000000000002
9 5.654866776461628 0.12274
10 6.283185307179586 0.09402
deltas:
 [0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628, 6.283185307179586]
costs:
 [0.09408, 0.12778, 0.20297, 0.29556, 0.36757000000000006, 0.3901300000000001, 0.36123000000000005, 0.29033, 0.19801000000000002, 0.12274, 0.09402]


In [2]:
# 参数化量子线路等价性验证
from mindquantum import Circuit, apply, ParameterResolver
from cqlib import TianYanPlatform, QuantumLanguage
import numpy as np
import json
np.random.seed(1)

def compute_cost(result:dict):
    res = json.loads(result['probability'])
    items = ['0111', '1011', '1101', '1110', '1010', '0101']
    cost = sum([res.get(item, 0) for item in items])
    return cost

def hardware_run(circs:list[Circuit], shots:int=1000):
    """向云平台批量提交线路信息, 并获取返回结果"""
    tasks = [circ.to_qcis(parametric=False) for circ in circs]
    query_list = platform.submit_experiment(circuit=tasks,
                                            language=QuantumLanguage.QCIS,
                                            num_shots=shots)
    results = platform.query_experiment(query_id=query_list,
                                            max_wait_time=60000,
                                            sleep_time=5)
    costs = []
    for i in range(len(circs)):
        costs.append(compute_cost(results[i]))
    return costs

def func_cost(circuit:Circuit, pr_delta_alphas:dict):
    """求给定 ansatz 参数下，不同 encoder 参数下的损失函数值, values_a: ansatz 的参数"""
    circuit = circuit.apply_value(pr_delta_alphas)
    
    def sampling(values_beta):
        """values_beta: beta 的参数"""
        circs = []
        for beta in values_beta:
            circs.append(circuit.apply_value({"beta":beta}))
        costs = hardware_run(circs)
        return np.array(costs).mean()
    return sampling

# 简单编码线路  
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

beta = ParameterResolver("beta")
ref_circ = Circuit().ry(beta/2, 1).h(1).z(1, 0)
ref_circ.ry(beta/2,1).z(1,0).h(1)

gen_circ = Circuit().h(3).z(3,2)
gen_circ.ry(beta/2,3).z(3,2).h(3).ry(beta/2, 3).ry("delta",3)

disc_weight = np.array([0.1692159, 0.1617644, -0.11142179, 0.05916444]) # 一个训练好的实例

disc_circ = Circuit()
disc_circ.h(0).z(2,0).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ
circ.measure_all()

physical_qubits = [10, 16, 4, 11] # 比特映射
# physical_qubits = [6, 7, 18, 19] # 比特映射
circ = apply(circ, physical_qubits)

# 编码器随机参数
sample_num_alpha = 10
alphas = np.random.rand(sample_num_alpha, len(encoder.params_name)) * np.pi * 2

# 参数化线路随机参数
sample_num_beta = 10
betas = np.random.rand(sample_num_beta) * np.pi * 2

login_key = "lABSAqU6oBZCM9wc6owgpVQ0Jdnjr4JkQZXnHjMxck0="
machine_name="tianyan176-2"
platform = TianYanPlatform(login_key=login_key, 
                           machine_name=machine_name)

costs_delta = []
deltas = []
for i in range(11): # delta
    delta = i * 2 * np.pi / 10
    deltas.append(delta)
    pr = {"delta":delta}
    costs_alphas = []
    for alpha in alphas: # alpha
        pr.update(dict(zip(encoder.params_name, alpha)))
        sampling = func_cost(circ, pr) # type:ignore
        cost = sampling(betas)
        costs_alphas.append(cost)
    cost_delta = sum(costs_alphas) / len(costs_alphas)
    print(i, cost_delta)
    costs_delta.append(cost_delta)

print("deltas:\n", deltas)
print("costs_delta:\n", costs_delta)

deltas:
 [0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628, 6.283185307179586]
costs_delta:
 [0.10681, 0.13075, 0.19740000000000002, 0.27652, 0.34281, 0.37661, 0.34796, 0.27671, 0.18785000000000002, 0.11632, 0.09168]


In [ ]:
# 变分量子线路优化：训练生成器
from mindquantum.core import Circuit, apply
from cqlib import TianYanPlatform, QuantumLanguage
from copy import deepcopy
import json
import numpy as np
import mindspore as ms
from mindspore import nn
from mindspore.common.parameter import Parameter

def update(dict0:dict, dict1:dict):
    """合并两个字典"""
    return {**dict0, **dict1}

def compute_cost(result:dict):
    res = json.loads(result['probability'])
    items = ['0111', '1011', '1101', '1110', '1010', '0101']
    cost = sum([res.get(item, 0) for item in items])
    return cost

def hardware_run(circs:list[Circuit], shots:int=5000):
    """向云平台批量提交线路信息, 并获取返回结果"""
    tasks = [circ.to_qcis(parametric=False) for circ in circs]
    query_list = platform.submit_experiment(circuit=tasks,
                                            language=QuantumLanguage.QCIS,
                                            num_shots=shots)
    results = platform.query_experiment(query_id=query_list,
                                            max_wait_time=600,
                                            sleep_time=2)
    costs = []
    for i in range(len(circs)):
        costs.append(compute_cost(results[i]))
    return costs

def get_cost_with_grads(circuit:Circuit, values_a:np.ndarray):
    """根据参数移位法计算当前参数的梯度"""
    names_e = circuit.encoder_params_name
    names_a = circuit.ansatz_params_name
    h = np.pi / 2
    def grad_ops(values_e:np.ndarray):
        """values_e: encoder 的参数; values_a: ansatz 的参数"""
        circs = []
        for value_e in values_e:
            pr_e = dict(zip(names_e, value_e))
            pr_0 = update(pr_e, dict(zip(names_a, values_a)))
            circs.append(circuit.apply_value(pr_0)) # 前 batch_size 个线路为求期望值数据
        # 格式为:
        # a1p, a1n, a2p, a2n ... 样本 0
        # a1p, a1n, a2p, a2n ... 样本 1
        # ...
        for value_e in values_e:
            # 计算参数移位法所需的损失函数值
            pr_e = dict(zip(names_e, value_e))
            for i in range(len(values_a)):
                values_a_p = deepcopy(values_a)
                values_a_n = deepcopy(values_a)

                values_a_p[i] += h
                pr_a_p = update(pr_e, dict(zip(names_a, values_a_p)))
                circs.append(circuit.apply_value(pr_a_p))

                values_a_n[i] -= h
                pr_a_n = update(pr_e, dict(zip(names_a, values_a_n)))
                circs.append(circuit.apply_value(pr_a_n))

        costs = hardware_run(circs)
        cost = np.mean(costs[:batch_size])
        costs = costs[batch_size:]
        costs = ms.Tensor(costs).reshape(batch_size, -1)
        cost_p = costs[:,0::2].mean(axis=0)
        cost_n = costs[:,1::2].mean(axis=0)
        grads = (cost_p - cost_n) / 2
        return cost, grads
    return grad_ops

# 简单编码线路  
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 参考线路
ref_circ = Circuit().x(0).x(1).z(1,0).x(0).x(1)

# 生成线路
gen_circ = Circuit().rz("beta0",2).rz("beta1",3).z(3,2)
gen_circ.as_ansatz()

# 鉴别器
disc_weight = np.array([0.1692159, 0.1617644, -0.11142179, 0.05916444]) # 一个训练好的实例
disc_circ = Circuit()
disc_circ.h(0).z(2, 0).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ # type:ignore
circ.measure_all()

physical_qubits = [10, 16, 4, 11] # 比特映射
circ = apply(circ, physical_qubits)

login_key = "lABSAqU6oBZCM9wc6owgpVQ0Jdnjr4JkQZXnHjMxck0="
machine_name="tianyan176-2"
platform = TianYanPlatform(login_key=login_key, 
                           machine_name=machine_name)

beta = Parameter(ms.Tensor([0.0, 0.0]), name="beta") 
optim = nn.Adam([beta], learning_rate=0.1)

steps_num = 51
batch_size = 4
alphas = np.random.rand(steps_num, batch_size, len(encoder.params_name)) * np.pi * 2

min_cost = 1
costs = []

for i in range(steps_num):
    grad_ops = get_cost_with_grads(circ, beta) # type:ignore
    cost, grads = grad_ops(ms.Tensor(alphas[i]))
    if cost < min_cost:
        best_gen_weight = beta.asnumpy()
        min_cost = cost
    print("step:", i, "beta", beta.asnumpy(), "cost:", cost, "grads:", grads.asnumpy())
    costs.append(cost)
    optim((grads,))

step: 0 beta [0. 0.] cost: 0.43304999999999993 grads: [0.005575   0.00605001]
step: 1 beta [-0.09999432 -0.09999478] cost: 0.38565000000000005 grads: [0.006825 0.001325]
step: 2 beta [-0.20000902 -0.18137075] cost: 0.39865 grads: [0.02595 0.0149 ]
step: 3 beta [-0.28534615 -0.26449078] cost: 0.42255 grads: [0.01615    0.00957501]
step: 4 beta [-0.37507743 -0.35270458] cost: 0.36535 grads: [0.011125   0.02007499]
step: 5 beta [-0.46468425 -0.44346046] cost: 0.39685000000000004 grads: [0.0704     0.01672499]
step: 6 beta [-0.54509914 -0.5374243 ] cost: 0.41280000000000006 grads: [0.04727499 0.01445001]
step: 7 beta [-0.63140297 -0.63303304] cost: 0.3359 grads: [0.04072499 0.03795001]
step: 8 beta [-0.7210898 -0.7267023] cost: 0.37644999999999995 grads: [0.07712501 0.021375  ]
step: 9 beta [-0.81397414 -0.82221705] cost: 0.3582 grads: [0.0388     0.06524999]
step: 10 beta [-0.907454   -0.91448224] cost: 0.31625 grads: [0.09487499 0.036625  ]
step: 11 beta [-1.003614  -1.0094529] cost: 0.2

In [10]:
# 变分量子线路优化：检验生成器效果
from mindquantum.core import Circuit, apply
from cqlib import TianYanPlatform, QuantumLanguage
import json
import numpy as np

def update(dict0:dict, dict1:dict):
    """合并两个字典"""
    return {**dict0, **dict1}

def compute_cost(result:dict):
    res = json.loads(result['probability'])
    items = ['0111', '1011', '1101', '1110', '1010', '0101']
    cost = sum([res.get(item, 0) for item in items])
    return cost
    
def hardware_run(circs:list[Circuit], shots:int=1000):
    """向云平台批量提交线路信息, 并获取返回结果"""
    tasks = [circ.to_qcis(parametric=False) for circ in circs]
    query_list = platform.submit_experiment(circuit=tasks,
                                            language=QuantumLanguage.QCIS,
                                            num_shots=shots)
    results = platform.query_experiment(query_id=query_list,
                                            max_wait_time=60000,
                                            sleep_time=2)
    costs = []
    for i in range(len(circs)):
        costs.append(compute_cost(results[i]))
    return costs

def func_cost(circuit:Circuit, values_a):
    """求给定 ansatz 参数下，不同 encoder 参数下的损失函数值, values_a: ansatz 的参数"""
    names_e = circuit.encoder_params_name
    names_a = circuit.ansatz_params_name
    pr_a = dict(zip(names_a, values_a))
    circuit = circuit.apply_value(pr_a)
    
    def sampling(values_e):
        """values_e: encoder 的参数"""
        circs = []
        for values in values_e:
            pr_e = dict(zip(names_e, values))
            circs.append(circuit.apply_value(pr_e))
        costs = hardware_run(circs)
        return np.array(costs).mean()
    return sampling

# 简单编码线路  
encoder = Circuit()
encoder.ry("alpha0", 0).rz("alpha1", 0).ry("alpha2", 0)
encoder.ry("alpha3", 1).rz("alpha4", 1).ry("alpha5", 1)
encoder.ry("alpha0", 2).rz("alpha1", 2).ry("alpha2", 2)
encoder.ry("alpha3", 3).rz("alpha4", 3).ry("alpha5", 3)
encoder.as_encoder()

# 参考线路
ref_circ = Circuit().x(0).x(1).z(1,0).x(0).x(1)

# 生成线路
gen_circ = Circuit().rz("beta0",2).rz("beta1",3).z(3,2)
gen_circ.as_ansatz()

# 鉴别器
disc_weight = np.array([0.1692159, 0.1617644, -0.11142179, 0.05916444]) # 一个训练好的实例
disc_weight = np.array([0., 0., 0., 0.])
disc_circ = Circuit()
disc_circ.h(0).z(2, 0).rz("theta0",0).rz("theta1",2)
disc_circ.h(0).h(2).h(1).z(3,1).rz("theta2",1).rz("theta3",3).h(1).h(3)
disc_circ = disc_circ.apply_value(dict(zip(disc_circ.params_name, disc_weight))) # 训练好的鉴别器

circ = encoder + ref_circ + gen_circ + disc_circ # type:ignore
circ.measure_all()

physical_qubits = [10, 16, 4, 11] # 比特映射
circ = apply(circ, physical_qubits)

login_key = "lABSAqU6oBZCM9wc6owgpVQ0Jdnjr4JkQZXnHjMxck0="
machine_name="tianyan176-2"
platform = TianYanPlatform(login_key=login_key, 
                           machine_name=machine_name)


sample_num = 100
alphas = np.random.rand(sample_num, len(encoder.params_name)) * np.pi * 2

# 用全零参数
beta = np.array([0., 0.])
sampling = func_cost(circ, beta) # type:ignore
cost_0 = sampling(alphas[:50])
cost_1 = sampling(alphas[50:])
print((cost_0 + cost_1) / 2)


# 用训练好的参数
beta = np.array([-2.8470423, -3.0294402])
sampling = func_cost(circ, beta) # type:ignore
cost_0 = sampling(alphas[:50])
cost_1 = sampling(alphas[50:])
print((cost_0 + cost_1) / 2)

0.39719000000000004
0.07491
